# Homework Lecture 1

## Preliminaries

### Imports

In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

import sys 
sys.path.append("../..")
import E4525_ML.mnist as mnist

### Data Directories 

Create a directory with the path below

In [4]:
raw_data_dir="../../raw/MNIST"
data_dir="../../data/Introduction"


<div class="alert alert-block alert-info"> Problem 0 </div>
Download the file `data.csv` and save it fo the directory `data_dir`

## Numpy Exercises

### Read and Inspect the Data 

<div class="alert alert-block alert-info"> Problem 1.1 </div>
Read the contents of `data.csv` into a panda's dataframe

In [5]:
data=pd.read_csv(data_dir+"/data.csv")

<div class="alert alert-block alert-info"> Problem 1.2 </div>
What are the name of the columns?

In [6]:
data.columns

Index(['X', 'Y'], dtype='object')

<div class="alert alert-block alert-info"> Problem 1.3 </div>
How many rows are there?

In [7]:
len(data)

1000

<div class="alert alert-block alert-info"> Problem 1.4 </div>
List the unique values of column "X", and of column "Y"

In [8]:
print("X",data["X"].unique())
print("Y",data["Y"].unique())

X ['A' 'B' 'C' 'D']
Y ['a' 'c' 'd' 'b' 'e']


<div class="alert alert-block alert-info"> Problem 1.5 </div>
Use [pandas.Series.value_counts](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.value_counts.html) function to get the counts of each possible value  of X and Y variables

In [9]:
hist_X=data["X"].value_counts()
hist_X

A    383
B    293
C    279
D     45
Name: X, dtype: int64

In [10]:
hist_Y=data["Y"].value_counts()
hist_Y

c    349
a    312
b    148
d    132
e     59
Name: Y, dtype: int64

### Basic Usage of Numpy

<div class="alert alert-block alert-info"> Problem 2.1 </div>

Use panda's function [get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) to
* create a variable named `data_ZX` with one row per row on the original data, and one-hot encoded `X` variable values as columns
* create a varialbe named `data_ZY` with one row per row on the original data, and one-hot encoded `Y` variable values as columns

In [11]:
data_ZX=pd.get_dummies(data["X"])
data_ZX.shape

(1000, 4)

In [12]:
data_ZY=pd.get_dummies(data["Y"])
data_ZY.shape

(1000, 5)

<div class="alert alert-block alert-info"> Problem 2.2 </div>
Using the dataframe function `as_matrix()`  create `numpy` arrays `ZX` and `ZY` with the occurrence data for `X` and `Y` variables

In [15]:
ZX=data_ZX.values
ZX.shape

(1000, 4)

In [17]:
ZX

array([[1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       ...,
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0]], dtype=uint8)

In [16]:
ZY=data_ZY.values
ZY.shape

(1000, 5)

<div class="alert alert-block alert-info"> Problem 2.3 </div>
Save the column names of `data_ZX` in a variable named `X_values`, create also `Y_values`

In [ ]:
X_values=data_ZX.columns
X_values

In [ ]:
Y_values=data_ZY.columns
Y_values

### Covariance calculation

From now on, work **only** with the variables `ZX`, and `ZY`, `X_values` and `Y_values`

<div class="alert alert-block alert-info"> Problem 3.1 </div>
Show the shape of `ZX` and `ZY` variables

In [ ]:
print(ZX.shape,ZY.shape)

<div class="alert alert-block alert-info"> Problem 3.2 </div>
* using `ZX`  compute the number of occurrences of each possible value of the variable X
* using `ZY`  compute the number of occurrences of each possible value of the variable Y

for each posible value of 'X' print the value (a string), and how may times it occurs on the data (an integer).
Do the same for 'Y'

In [ ]:
X_counts=ZX.sum(axis=0)
Y_counts=ZY.sum(axis=0)

In [ ]:
for i in range(len(X_values)):
    print(X_values[i],X_counts[i])

In [ ]:
for i in range(len(Y_values)):
    print(Y_values[i],Y_counts[i])

<div class="alert alert-block alert-info"> Problem 3.3</div>
Compute the empirical (sample) probabilities of the  values of `X` and `Y`.
$$
    \mu^X_d = \frac{1}{N} \sum_i Z^X_{i,k}
$$
and similarly for $\mu^Y_k$

Store the results on variables `muX` and `muY`

In [ ]:
N=len(ZX)
muX=X_counts/N
muY=Y_counts/N
print("muX",muX)
print("muY",muY)

<div class="alert alert-block alert-info"> Problem 3.4 </div>
Compute the covariance matrix of `ZX` and `ZY` values defined as 
$$
    \sigma^{X}_{d,d'} = \frac{1}{N}\sum_i Z^X_{i,d}Z^X_{i,d'}- \mu^X_d \mu^X_{d'}
$$
and similarly for $\sigma^Y_{k,k'}$

In [ ]:
sigX=np.mean(ZX[:,:,np.newaxis]*ZX[:,np.newaxis,:],axis=0) - muX[:,np.newaxis]*muX[np.newaxis,:]
sigX

In [ ]:
sigY=np.mean(ZY[:,:,np.newaxis]*ZY[:,np.newaxis,:],axis=0) - muY[:,np.newaxis]*muY[np.newaxis,:]
sigY

<div class="alert alert-block alert-info"> Problem 3.5 </div>
Compute the convariance matrix between `ZX` and `ZY` values defined as
$$
    \sigma^{Y,X}_{k,d} = \frac{1}{N}\sum_i Z^Y_{i,k}Z^X_{i,d}- \mu^Y_k\mu^X_d 
$$

In [ ]:
sigXY=np.mean(ZY[:,:,np.newaxis]*ZX[:,np.newaxis,:],axis=0) - muY[:,np.newaxis]*muX[np.newaxis,:]
sigXY

### Exponential weighted covariance

Some times, we want to give more weight so some observations $i$  others.

For example, We could interpret the sample index $i$, as time, and we want to weight more strongly observations close to the present more 
strongly than ones far in the pass.

<div class="alert alert-block alert-info"> Problem 4.1 </div>
Generate an array $W$ of shape $(N,)$ such that
$$
    W_i =\frac{ e^{-\lambda i}}{\sum_{j} e^{-\lambda j}}
$$
with $\lambda = 4/N$

Verify that 
$$
     \sum_i W_i = 1
$$
[Hints]
* Try not write a python for loop (too slow). 
* It is probably easier to break the implementation into  two or three intermediate steps.
* The function [`numpy.arange`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.arange.html) will probably be useful.

In [ ]:
lambda0=4/N
I=np.arange(N)
eI=np.exp(-lambda0*I)
eISum=eI.sum()
W=eI/eISum
print(W.sum(),W[:10])

<div class="alert alert-block alert-info"> Problem 4.2 </div>
Compute the $X$ and $Y$ weighted means defined as
$$
    \mu^X_{W,d} = \sum_i W_i Z^X_{i,d}
$$
and similarly for $Y$

In [ ]:
muXW=np.dot(W,ZX)
muYW=np.dot(W,ZY)
print("muXW",muXW)
print("muYW",muYW)

In [ ]:
# another possible solution
muXW=np.sum(W[:,np.newaxis]*ZX,axis=0)
muYW=np.sum(W[:,np.newaxis]*ZY,axis=0)
print("muXW",muXW)
print("muYW",muYW)

<div class="alert alert-block alert-info"> Problem 4.3 </div>
Compute the exponentially weighted  covariance matrix defined as
$$
    \sigma^{Y,X}_{W,\,k,d} = \sum_i W_i (Z^Y_{i,k} - \mu^Y_{W,k})(Z^X_{i,k} - \mu^X_{W,k})
$$

In [ ]:
dZX= ZX-muXW
dZY= ZY-muYW
Cov=dZY[:,:,np.newaxis]*dZX[:,np.newaxis,:]
sigXYW=np.sum(W[:,np.newaxis,np.newaxis]*Cov,axis=0)
print(sigXYW)

## Analyzing MNIST with Numpy

<div class="alert alert-block alert-info"> Problem 5.0 </div>
Got to MNIST dataset's [web site](http://yann.lecun.com/exdb/mnist/)'.

Download the floowing four files, and save them with those names in the raw data directory [raw/MINST](../../raw/MNIST)
* `train-images-idx3-ubyte.gz`:  training set images
* `train-labels-idx1-ubyte.gz`:  training set labels
* `t10k-images-idx3-ubyte.gz`:   test set images
* `t10k-labels-idx1-ubyte.gz`:   test set labels

Make sure not the decompress, or modify the files in any way.


<div class="alert alert-block alert-info"> Problem 5.1 </div>
Using `E54525_ML.mnist.read_images` function read the train images in file `train-images-idx3-ubyte.gz` into a variable named
`images`. 

images will be a tensor (`np.array` of floating point numbers) with three dimensions $N\times R \times C$ where
1. $N$ is the number of images
2. $R$ is the number of column pixels in an image
3. $C$ is the number of row pixels.

Show whe shape of the `images` variable

In [ ]:
images_filename=raw_data_dir+"/train-images-idx3-ubyte.gz"
images=mnist.read_images(images_filename)
print(images.shape)

In [ ]:
N=images.shape[0]
R=images.shape[1]
C=images.shape[2]
print("N",N)
print("R",R)
print("C",C)

<div class="alert alert-block alert-info"> Problem 5.2 </div>
Read the labels from file `train-labels-idx1-ubyte.gz`, save them in a variable named labels
What is the shape of  label's variable?

In [ ]:
labels_filename=raw_data_dir+"/train-labels-idx1-ubyte.gz"
labels=mnist.read_labels(labels_filename)
labels.shape

<div class="alert alert-block alert-info"> Problem 5.3 </div>
Use [`matplotlib.pyplot.imshow`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.imshow.html?highlight=matplotlib%20pyplot%20imshow#matplotlib.pyplot.imshow) to display the first three images on the MNIST dataset

Se the plot title to each image to the corresponding label

[HINT] set the color map to `gray` for best results

In [ ]:
label=int(labels[0])
plt.title('Digit {0}'.format(label))
plt.imshow(images[0],cmap="gray")

In [ ]:
label=int(labels[1])
plt.title('Digit {0}'.format(label))
plt.imshow(images[1], cmap='gray')

In [ ]:
label=int(labels[2])
plt.title('Digit {0}'.format(label))
plt.imshow(images[2], cmap='gray')

<div class="alert alert-block alert-info"> Problem 5.3 </div>
Compute the average image (averaged over all the samples).
use `imshow` to plot the result.

[HINT] this is just averaging over the first axis of the images array

In [ ]:
mean_image=np.mean(images,axis=0)
print(mean_image.shape)
plt.title("Average pixel intensities")
plt.imshow(mean_image,cmap="gray")

In [ ]:
def one_hot_encode(labels):
    N=len(labels)
    K=labels.max()+1
    Z=np.zeros((len(labels),K))
    Z[np.arange(N),labels]=1
    return Z

<div class="alert alert-block alert-info"> Problem 5.4 </div>
Use the function `one_hot_encode` defined above to create a variable `Z` with the one hot encoding of the labels.

Show the shape of `Z`

In [ ]:
Z=one_hot_encode(labels)
Z.shape

<div class="alert alert-block alert-info"> Problem 5.5 </div>
Create the rank four tensor
$$
T_{i,k,r,c} = Z_{i,k}  \textrm{images}_{i,r,c}
$$


In [ ]:
T=Z[:,:,np.newaxis,np.newaxis]*images[:,np.newaxis,:,:]
T.shape

<div class="alert alert-block alert-info"> Problem 5.6 </div>
Average $T$ over the sample index $i$, save to a variable named `average`

In [ ]:
average=T.mean(axis=0)
average.shape

<div class="alert alert-block alert-info"> Problem 5.7</div>
What does `average[0]` represent? 

and `average[5]`?

`average[0]` is the average intensity of  the pixels for all the images labeled 0 (scaled by 1/K=0.1).

`average[5]` is the average intensity of the pixels for all the images labeled 5 (scaled by 1/K=0.1).

<div class="alert alert-block alert-info"> Problem 5.8 </div>
use `imshow` to plot `average[0]` and `average[5]`

In [ ]:
plt.title("Average Digit 0")
plt.imshow(average[0],cmap="gray")

In [ ]:
plt.title("Average Digit 5")
plt.imshow(average[5],cmap="gray")